In [ ]:
from socioFetcher import Downloader, GeoDataFrame, Config
from ipyleaflet import Map, basemaps, basemap_to_tiles, GeoData, GeoJSON
import ipyleaflet
from branca.colormap import linear
import geopandas 
import requests 
from shapely.geometry import shape
import json

from ipywidgets import IntSlider, HTML

In [ ]:
dataset = ["BEA", "ACS"]
fipsList = ["26161", "26163"]
downloader = Downloader(dataset, fipsList)
downloader.download()

In [ ]:
r = requests.get("https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/State_County/MapServer/1/query?where=STATE%3D%2226%22+&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=GEOID%2C+STATE%2C+COUNTY%2C+NAME&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentsOnly=false&datumTransformation=&parameterValues=&rangeValues=&f=geojson")




In [ ]:
m = Map(
    layers=(basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2017-04-08"), ),
    center=(42.124026, -84.102547),
    zoom=4
)

In [ ]:
geo_json = GeoJSON(data=r.json(),
                   style = {
                       'color': 'green',
                       'opacity':1,
                       'weight':1.9,
                       'dashArray':'9',
                       'fillOpacity':0.1})

m.add_layer(geo_json)

In [ ]:
chorLyr = ipyleaflet.Choropleth(
    geo_data=geo_json.data
)

In [ ]:
m

In [ ]:
def get_county_geojson(fipsList, outFields):
    where = ""
    for i, fips in enumerate(fipsList):
        where += f"(STATE='{fips[:2]}' AND COUNTY='{fips[2:]}')"
        if i != len(fipsList)-1:
            where += " OR "
    params = {
        "where": where,
        "outFields": outFields,  #"GEOID, STATE, COUNTY, NAME",
        "f":"geojson",
        "text": "",
        "objectIds": "",
        "time": "",
        "geometry": "",
        "geometryType": "esriGeometryEnvelope",
        "inSR": "",
        "spatialRel": "esriSpatialRelIntersects",
        "relationParam": "",
        "returnGeometry": True,
        "returnTrueCurves": False,
        "maxAllowableOffset": "",
        "geometryPrecision": "",
        "outSR": "",
        "returnIdsOnly": False,
        "returnCountOnly": False,
        "orderByFields": "",
        "groupByFieldsForStatistics": "",
        "outStatistics": "",
        "returnZ": False,
        "returnM": False,
        "gdbVersion": "",
        "returnDistinctValues": False,
        "resultOffset": "",
        "resultRecordCount": "",
        "queryByDistance": "",
        "returnExtentsOnly": False,
        "datumTransformation": "",
        "parameterValues": "",
        "rangeValues": ""
    }
    r = requests.get(
        "https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/State_County/MapServer/1/query",
        params=params
    )
    print(r.url)
    
    return r.json()

In [ ]:
def add_id(geojson, idname):
    for i, feature in enumerate(geojson["features"]):
        geojson["features"][i]["id"] = geojson["features"][i]["properties"][idname]
    return geojson
        
geojson = add_id(geo_json.data, "GEOID")

In [ ]:
data = get_county_geojson(["26161", "26163"], "GEOID")

In [ ]:
data = add_id(data, "GEOID")
choro_data = {
          "26161": 5,
          "26163": 10
      }
m = Map(
    layers=(ipyleaflet.basemap_to_tiles(ipyleaflet.basemaps.CartoDB.Positron), ),
    center=(42.124026, -84.102547),
    zoom=9
)
chron = ipyleaflet.Choropleth(geo_data=data,
                              choro_data=choro_data,
                              colormap=linear.YlOrRd_04,
                              name="ACS",
                               style = {
                                   'opacity':1,
                                   'weight':1.9,
                                   'dashArray':'9',
                                   'fillOpacity':0.5})
def handle_click(**kwargs):
    chron.style = {'opacity': 1, 'weight': 1.9, 'dashArray': '9', 'fillOpacity': 1}
    chron.choro_data =  {
          "26161": 20,
          "26163": 10
      }
    data_box.value = "new data"
    
chron.on_click(handle_click)

year_slider = IntSlider(description='Year:', min=2010, max=2017, value=2017)
year_widget = ipyleaflet.WidgetControl(widget=year_slider, position='topright')

data_box = HTML(
    value="click to show data"
)
data_widget = ipyleaflet.WidgetControl(widget=data_box, position='topright')


m.add_layer(chron)
m.add_control(ipyleaflet.LayersControl())
m.add_control(ipyleaflet.FullScreenControl())
m.add_control(year_widget)
m.add_control(data_widget)
m

In [1]:
from socioFetcher.mapview import MapView
mapview = MapView("", "", "", zoom=10)
mapview.show()

Loading Data for Wayne,MI
Loading Data for Washtenaw,MI
Loading Data for Wayne,MI
Loading Data for Washtenaw,MI


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …